In [1]:
from modules.base import *

#### 요약 사용하기

---

In [2]:
LIMIT_LENGTH = 12 # 대화기록이 특정길이 넘어갈경우 일부삭제

In [3]:
class State(MessagesState):
    summary: str

def node_conversation(state: State, 
                      config: RunnableConfig):
    
    summary = state.get("summary", "")
    if summary:
        system_message = f"다음은 지금까지의 대화 요약입니다. {summary}\n\n"
        messages = [SystemMessage(content=system_message)] + state["messages"]
    else:
        messages = state["messages"]
    response = llm.invoke(messages, config)
    return {"messages": response}

def node_should_continue(state: State):    
    messages = state["messages"]
    if len(messages) >= LIMIT_LENGTH:
        print(f"{YELLOW}\n대화 길이가 {LIMIT_LENGTH} 이상 ({LIMIT_LENGTH//2}번 이상 대화 주고받음) 이므로 요약 노드로 이동합니다.\n{RESET}")
        return "node_summarize_conversation"
    return END

def node_summarize_conversation(state: State):
    summary = state.get("summary", "")
    if summary:
        summary_message = (
            f"다음은 지금까지의 대화 요약입니다. {summary}\n\n"
            "새로운 메시지를 고려하여 요약 내용을 업데이트하세요."
        )
    else:
        summary_message = "위 내용을 요약하세요."
    messages = state["messages"] + [HumanMessage(content=summary_message)]
    response = llm.invoke(messages)
    delete_messages = [RemoveMessage(id=m.id) for m in state["messages"][:LIMIT_LENGTH//2]]
    return {"summary": response.content, "messages": delete_messages}

workflow = StateGraph(State)
workflow.add_node("node_conversation", node_conversation)
workflow.add_node("node_summarize_conversation", node_summarize_conversation)
workflow.add_edge(START, "node_conversation")
workflow.add_conditional_edges("node_conversation", node_should_continue)
workflow.add_edge("node_summarize_conversation", END)
ShortTermMemory = MemorySaver()
graph = workflow.compile(checkpointer=ShortTermMemory)

In [4]:
config = {"configurable": {"thread_id": "first_chat", 
                           "user_id": "changwoo"}}

In [5]:
graph.invoke({"messages":"안녕 나는 창우라고해"}, config=config)

{'messages': [HumanMessage(content='안녕 나는 창우라고해', additional_kwargs={}, response_metadata={}, id='b0a0e16e-f320-4669-9f64-956e96a272ab'),
  AIMessage(content='안녕하세요, 창우님! 만나서 반갑습니다. 어떻게 도와드릴까요?', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 21, 'prompt_tokens': 14, 'total_tokens': 35, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-2024-08-06', 'system_fingerprint': 'fp_703d4ff298', 'finish_reason': 'stop', 'logprobs': None}, id='run-064de447-7b46-4c4e-81e3-d4553a5be302-0', usage_metadata={'input_tokens': 14, 'output_tokens': 21, 'total_tokens': 35, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})]}

In [6]:
graph.invoke({"messages":"나는 올해 30살이고, 현재 부산대학교에서 박사과정을 하고있어."}, config=config)

{'messages': [HumanMessage(content='안녕 나는 창우라고해', additional_kwargs={}, response_metadata={}, id='b0a0e16e-f320-4669-9f64-956e96a272ab'),
  AIMessage(content='안녕하세요, 창우님! 만나서 반갑습니다. 어떻게 도와드릴까요?', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 21, 'prompt_tokens': 14, 'total_tokens': 35, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-2024-08-06', 'system_fingerprint': 'fp_703d4ff298', 'finish_reason': 'stop', 'logprobs': None}, id='run-064de447-7b46-4c4e-81e3-d4553a5be302-0', usage_metadata={'input_tokens': 14, 'output_tokens': 21, 'total_tokens': 35, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}}),
  HumanMessage(content='나는 올해 30살이고, 현재 부산대학교에서 박사과정을 하고있어.', additional_kwargs={}, response_metadata={}, 

In [7]:
graph.invoke({"messages":"내 전공은 인공지능이고, 요즘 LLM 분야에 관심이 많아."}, config=config)

{'messages': [HumanMessage(content='안녕 나는 창우라고해', additional_kwargs={}, response_metadata={}, id='b0a0e16e-f320-4669-9f64-956e96a272ab'),
  AIMessage(content='안녕하세요, 창우님! 만나서 반갑습니다. 어떻게 도와드릴까요?', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 21, 'prompt_tokens': 14, 'total_tokens': 35, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-2024-08-06', 'system_fingerprint': 'fp_703d4ff298', 'finish_reason': 'stop', 'logprobs': None}, id='run-064de447-7b46-4c4e-81e3-d4553a5be302-0', usage_metadata={'input_tokens': 14, 'output_tokens': 21, 'total_tokens': 35, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}}),
  HumanMessage(content='나는 올해 30살이고, 현재 부산대학교에서 박사과정을 하고있어.', additional_kwargs={}, response_metadata={}, 

In [8]:
graph.invoke({"messages":"내 취미는 헬스장에서 운동하는거야"}, config=config)

{'messages': [HumanMessage(content='안녕 나는 창우라고해', additional_kwargs={}, response_metadata={}, id='b0a0e16e-f320-4669-9f64-956e96a272ab'),
  AIMessage(content='안녕하세요, 창우님! 만나서 반갑습니다. 어떻게 도와드릴까요?', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 21, 'prompt_tokens': 14, 'total_tokens': 35, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-2024-08-06', 'system_fingerprint': 'fp_703d4ff298', 'finish_reason': 'stop', 'logprobs': None}, id='run-064de447-7b46-4c4e-81e3-d4553a5be302-0', usage_metadata={'input_tokens': 14, 'output_tokens': 21, 'total_tokens': 35, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}}),
  HumanMessage(content='나는 올해 30살이고, 현재 부산대학교에서 박사과정을 하고있어.', additional_kwargs={}, response_metadata={}, 

In [9]:
graph.invoke({"messages":"내가 제일 좋아하는 음식은 돼지고기고, 싫어하는 음식은 딱히 없어."}, config=config)

{'messages': [HumanMessage(content='안녕 나는 창우라고해', additional_kwargs={}, response_metadata={}, id='b0a0e16e-f320-4669-9f64-956e96a272ab'),
  AIMessage(content='안녕하세요, 창우님! 만나서 반갑습니다. 어떻게 도와드릴까요?', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 21, 'prompt_tokens': 14, 'total_tokens': 35, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-2024-08-06', 'system_fingerprint': 'fp_703d4ff298', 'finish_reason': 'stop', 'logprobs': None}, id='run-064de447-7b46-4c4e-81e3-d4553a5be302-0', usage_metadata={'input_tokens': 14, 'output_tokens': 21, 'total_tokens': 35, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}}),
  HumanMessage(content='나는 올해 30살이고, 현재 부산대학교에서 박사과정을 하고있어.', additional_kwargs={}, response_metadata={}, 

In [10]:
graph.invoke({"messages":"내 가족관계는 아버지, 어머니, 형, 형수님, 조카 2명이야."}, config=config)


대화 길이가 12 이상 (6번 이상 대화 주고받음) 이므로 요약 노드로 이동합니다.



{'messages': [HumanMessage(content='내 취미는 헬스장에서 운동하는거야', additional_kwargs={}, response_metadata={}, id='b01efa7c-9911-4376-8b34-40cc1f3f2aec'),
  AIMessage(content='운동은 건강을 유지하고 스트레스를 해소하는 데 정말 좋은 취미죠! 헬스장에서 어떤 종류의 운동을 주로 하시나요? 웨이트 트레이닝, 유산소 운동, 혹은 다른 특별한 프로그램을 따르시는지 궁금하네요.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 64, 'prompt_tokens': 259, 'total_tokens': 323, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-2024-08-06', 'system_fingerprint': 'fp_703d4ff298', 'finish_reason': 'stop', 'logprobs': None}, id='run-e2d87550-80ac-4a60-9e58-881e60c73acd-0', usage_metadata={'input_tokens': 259, 'output_tokens': 64, 'total_tokens': 323, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}}),
  HumanMessage(conte

In [11]:
graph.invoke({"messages":"내 키는 179이고 체중은 84kg이야."}, config=config)

{'messages': [HumanMessage(content='내 취미는 헬스장에서 운동하는거야', additional_kwargs={}, response_metadata={}, id='b01efa7c-9911-4376-8b34-40cc1f3f2aec'),
  AIMessage(content='운동은 건강을 유지하고 스트레스를 해소하는 데 정말 좋은 취미죠! 헬스장에서 어떤 종류의 운동을 주로 하시나요? 웨이트 트레이닝, 유산소 운동, 혹은 다른 특별한 프로그램을 따르시는지 궁금하네요.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 64, 'prompt_tokens': 259, 'total_tokens': 323, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-2024-08-06', 'system_fingerprint': 'fp_703d4ff298', 'finish_reason': 'stop', 'logprobs': None}, id='run-e2d87550-80ac-4a60-9e58-881e60c73acd-0', usage_metadata={'input_tokens': 259, 'output_tokens': 64, 'total_tokens': 323, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}}),
  HumanMessage(conte